In [1]:
import numpy as np
from dataLoader import DataLoader

In [2]:
input_path = 'input/BioASQ-trainingDataset6b.json'
data = DataLoader(input_path)
data.load_ner_entities()

Loading ner entities from file: nerCache/BioASQ-trainingDataset6b.json


In [3]:
run retrieval_model.py

In [4]:
q = data[0]
scores = {}
for sentence, score in get_ranked_sentences(q.question, q.snippet_sentences, 'BM25'):
    scores[sentence] = score
for sentence, score in get_ranked_sentences(q.question, q.snippet_sentences, 'Indri'):
    scores[sentence] = [scores[sentence], score]

In [5]:
len(np.unique(q.snippet_sentences).tolist()), len(q.snippet_sentences)

(15, 20)

In [6]:
len(get_ranked_sentences(q.question, q.snippet_sentences, 'BM25'))

15

In [7]:
q.ranked_sentences()

[{'BM25': 2.3188813786746474,
  'Indri': 0.41422164245653387,
  'text': 'Hirschsprung disease (HSCR) is a multifactorial, non-mendelian disorder in which rare high-penetrance coding sequence mutations in the receptor tyrosine kinase RET contribute to risk in combination with mutations at other genes'},
 {'BM25': 1.9989343890926352,
  'Indri': 0.41446496181644588,
  'text': "Chromosomal and related Mendelian syndromes associated with Hirschsprung's disease"},
 {'BM25': 1.9633290741310481,
  'Indri': 0.41442363788439585,
  'text': "The majority of the identified genes are related to Mendelian syndromic forms of Hirschsprung's disease"},
 {'BM25': 1.8805000486638965,
  'Indri': 0.41432047225542812,
  'text': "In this study, we review the identification of genes and loci involved in the non-syndromic common form and syndromic Mendelian forms of Hirschsprung's disease"},
 {'BM25': 1.6718072220029261,
  'Indri': 0.41425099990895259,
  'text': 'On the basis of a skewed sex-ratio (M/F = 4/1) a